In [46]:
import pandas as pd

# Чтение файла xlsx
file_path = '1.xlsx'
df = pd.read_excel(file_path)
# Поиск строки, где находится заголовок "VK id"
vk_id_row = df[df.isin(["VK id"]).any(axis=1)].index[0]

# Установка строки с "VK id" в качестве заголовков столбцов
df.columns = df.iloc[vk_id_row]

# Удаление строк до заголовка
df = df.iloc[vk_id_row+1:].reset_index(drop=True)
# Отображение результата
# Отображение всех непустых данных из столбца "VK id"
vk_id_data = df["VK id"].dropna()

vk_id_data

0     290530655
2       1931147
3     207227130
4     253340861
6     138042735
7     172244589
8     168420440
9     711398942
12     65657314
14     50933461
16    198216820
18    268235974
Name: VK id, dtype: object

In [26]:
import requests
from vk_api import VkApi

In [27]:
token = ''

In [29]:
vk_session = VkApi(login='rsnek', token=token)

vk_session.auth(token_only=True)

In [30]:
vk = vk_session.get_api()

In [52]:
id_set = set(vk_id_data.values.ravel())
set2 = id_set.copy()

In [53]:
for vk_id in id_set:
    try:
        # Запрос к VK API для получения друзей пользователя
        current = vk.friends.get(user_id=vk_id, access_token=token)
        
        # Извлечение списка друзей (items) из JSON-ответа
        items = current.get('items', [])
        set2.update(items)
    except Exception as e:
        print(f"Ошибка при обработке VK ID {vk_id}: {e}")

# Список группы + друзья группы

Ошибка при обработке VK ID 65657314: [30] This profile is private
Ошибка при обработке VK ID 1931147: [30] This profile is private


In [55]:
set3=set2.copy()

In [58]:
import json

# Пустой словарь для хранения данных
friends_data = {}


for vk_id in set2:
    try:
        # Запрос к VK API для получения друзей пользователя
        current = vk.friends.get(user_id=vk_id, access_token=token)
        
        # Извлечение списка друзей (items) из JSON-ответа
        items = current.get('items', [])
        # Сохранение данных в словарь
        friends_data[vk_id] = items

        
    except Exception as e:
        ...


In [59]:
# Сохранение данных в JSON файл
with open('friends_data.json', 'w', encoding='utf-8') as f:
    json.dump(friends_data, f, ensure_ascii=False, indent=4)

print("Данные сохранены в 'friends_data.json'")

Данные сохранены в 'friends_data.json'


In [63]:
import networkx as nx
import matplotlib.pyplot as plt
import json

# Загрузка данных из JSON-файла
with open('friends_data.json', 'r', encoding='utf-8') as f:
    friends_data = json.load(f)

# Создание графа
G = nx.Graph()


In [68]:
edges_to_add = []
for user_id, friends in friends_data.items():
        for friend_id in friends:
            edges_to_add.append((user_id, friend_id))  # Сохраняем ребро
print(len(edges_to_add))

325863


In [69]:
all_friends = set(friend for friends in friends_data.values() for friend in friends)
print(len(all_friends))

260623


In [70]:
G.add_nodes_from(all_friends)
G.add_edges_from(edges_to_add)

In [71]:
nx.write_gexf(G, "0.gexf")